# Import Packages

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2 as cv
import random

from sklearn.model_selection import train_test_split
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, Flatten, MaxPool2D

# Step 2 - Getting data

In [ ]:
# ! mkdir ~/.kaggle
# ! cp /content/kaggle.json ~/.kaggle
# ! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# ! kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

In [ ]:
# ! unzip /content/gtsrb-german-traffic-sign.zip

In [ ]:
# ! rm /content/gtsrb-german-traffic-sign.zip

# Step 3 Data Preprocessing

In [ ]:
df = pd.read_csv('./Train.csv')

In [ ]:
# plotting 12 image to check datasets
plt.figure(figsize=(12, 12))
path = "/content/test/"
for i in range(1, 17):
  plt.subplot(4, 4, i)
  plt.tight_layout()
  img = plt.imread(f'{path}{random.choice(sorted(os.listdir(path)))}')
  plt.imshow(img)

In [ ]:
# # mean width and height
# height = []
# width = []
# path = "/content/test/"
# for i in range(50):
#   img = plt.imread(f'{path}{random.choice(sorted(os.listdir(path)))}')
#   height.append(img.shape[0])
#   width.append(img.shape[1])

# sum(height)/len(height), sum(width)/len(width)


In [ ]:
# IMG_SIZE = (50, 50)
# PATH = '/content/train'
# CLASSES = os.listdir(PATH)

In [ ]:
x = np.zeros((df.shape[0], 50, 50, 3), dtype='float32')
y = np.zeros((df.shape[0]), dtype='int32')

In [ ]:
for i in range(len(df)):
  y[i] = df['ClassId'][i]
  path = f"./{df['Path'][i]}"
  img = cv.imread(path)
  img = cv.resize(img, (50, 50))
  x_img = np.expand_dims(img, axis=0)
  x[i] = x_img / 255.0


In [ ]:
plt.imshow(x[600])

In [ ]:
# spliting the data
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=101)

In [ ]:
X_test.shape, y_test.shape

# Step 04 Building the model

In [ ]:
model = Sequential()

In [ ]:
# adding cnn layers

model.add(Conv2D(filters=64, kernel_size=(3, 3), input_shape=X_train.shape[1:],
                activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# 2nd
model.add(Conv2D(filters=32, kernel_size=(4, 4), activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# 3rd
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# flatting layers
model.add(Flatten())


# dense layers
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=43, activation='softmax'))

In [ ]:
# compile
opt = tf.keras.optimizers.Adam(0.0001)

model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

In [ ]:
# training the model
epochs=75
batch_size=128

history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
                    validation_data=(X_test, y_test)
                  )

In [ ]:
# evaluation of model

loss, accuracy = model.evaluate(X_train, y_train)

In [ ]:
print('the training score', accuracy)

In [ ]:
loss, accuracy, model.evaluate(X_test, y_test)

In [ ]:
print("The Testing Accuracy is:", accuracy)

# Building the Predictive model

In [ ]:
test_df = pd.read_csv('./Test.csv')

In [ ]:
def imageProcess(path):
  img = cv.imread(path)
  img = cv.resize(img,(50,50))
  img = img / 255
  return np.expand_dims(img, axis=0)

In [ ]:
for i in range(20):
  num = random.randint(1, 50)
  path = f"./{test_df['Path'][num]}"
  img = imageProcess(path)
  
  plt.imshow(img.reshape(50, 50, 3))
  plt.show()
  print(f"The prediction:",(np.argmax(model.predict(img))))
  print("The Actual", test_df['ClassId'][num])

